# Weather MCP Server Demo

This notebook demonstrates how to use the Weather MCP server to fetch weather data from Stormglass API.

## Features

- Get current weather conditions at any location
- Get weather forecasts (up to 240 hours)
- Get marine weather data (waves, swell, sea conditions)
- Get comprehensive weather data (combining standard + marine)
- Push notifications on successful weather queries

**Note**: Requires `STORMGLASS_API_KEY` in your `.env` file.


In [ ]:
# Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display

load_dotenv(override=True)


## Setup MCP Servers

We'll use two MCP servers:
1. **Weather Server** - For fetching weather data from Stormglass API
2. **Push Server** - For sending notifications on successful queries



In [ ]:
import os
from pathlib import Path

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Check if STORMGLASS_API_KEY is set
if not os.getenv("STORMGLASS_API_KEY"):
    print("⚠️ WARNING: STORMGLASS_API_KEY not found in environment variables.")
    print("Please set it in your .env file.")
else:
    print("✓ STORMGLASS_API_KEY found")

# Check if Pushover credentials are set
if not os.getenv("PUSHOVER_USER") or not os.getenv("PUSHOVER_TOKEN"):
    print("⚠️ WARNING: PUSHOVER_USER or PUSHOVER_TOKEN not found in environment variables.")
    print("Push notifications will not work. Please set them in your .env file.")
else:
    print("✓ Pushover credentials found")

weather_params = {
    "command": "uv",
    "args": ["run", "weather_server.py"]
}

push_params = {"command": "uv", "args": ["run", "push_server.py"]}

print("✓ MCP server parameters configured")


## Test the Weather MCP Server

Let's first check what tools are available:


In [ ]:
# List available tools from Weather MCP Server
async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as server:
    tools = await server.list_tools()
    print(f"Found {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.name}: {tool.description[:100]}...")


## Create a Weather Agent

This agent can fetch weather data and send notifications when queries are complete.


In [ ]:
instructions = """
You are a weather assistant that helps users get weather information.
You have access to comprehensive weather data from Stormglass API including:
- Current weather conditions (temperature, humidity, wind, etc.)
- Weather forecasts (up to 240 hours)
- Marine weather data (waves, swell, sea conditions)
- Comprehensive weather data (combining standard + marine)

When users ask about weather:
1. Use get_current_weather_tool for current conditions
2. Use get_weather_forecast_tool for forecasts
3. Use get_marine_weather_tool for coastal/marine locations
4. Use get_comprehensive_weather_tool for complete weather picture

IMPORTANT:
- You can use EITHER location names (e.g., "Nairobi, Kenya") OR coordinates (latitude, longitude)
- If user provides a location name, use it directly - the tool will automatically geocode it
- If user provides coordinates, use them directly
- If both are provided, coordinates take precedence
- Present weather data in a clear, organized format
- Include units (Celsius for temperature, m/s for wind speed, etc.)
- After completing a successful weather query, you MUST call the 'push' tool to send a push notification
  with a summary of the weather information retrieved (e.g., "Weather for Nairobi: 14.4°C, 70% humidity, wind 1.55 m/s")

Be helpful and provide detailed weather information when requested.
"""


## Example 1: Get Current Weather (Using Location Name)

Get current weather conditions using just the location name - geocoding is automatic!


In [ ]:
weather_query = "Get the current weather conditions for Nairobi, Kenya"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_current"):
            result = await Runner.run(agent, weather_query, max_turns=5)
            
        display(Markdown(result.final_output))


## Example 2: Get Weather Forecast (Using Location Name)

Get a weather forecast using just the location name.


In [ ]:
forecast_query = "Get a 48-hour weather forecast for Mombasa, Kenya"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_forecast"):
            result = await Runner.run(agent, forecast_query, max_turns=5)
            
        display(Markdown(result.final_output))


## Example 3: Get Marine Weather (Using Location Name)

Get marine weather data for a coastal location using just the location name.


In [ ]:
marine_query = "Get marine weather data including wave height and swell conditions for Mombasa, Kenya for the next 24 hours"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_marine"):
            result = await Runner.run(agent, marine_query, max_turns=5)
            
        display(Markdown(result.final_output))


## Example 4: Using Coordinates Directly

You can still use coordinates directly if you prefer or have them available.


In [ ]:
# Example: Using coordinates directly (still works!)
coords_query = "Get current weather for latitude 1.2921, longitude 36.8219"

async with MCPServerStdio(params=weather_params, client_session_timeout_seconds=60) as weather_server:
    async with MCPServerStdio(params=push_params, client_session_timeout_seconds=60) as push_server:
        agent = Agent(
            name="weather_assistant",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[weather_server, push_server]
        )
        
        with trace("weather_coordinates"):
            result = await Runner.run(agent, coords_query, max_turns=5)
            
        display(Markdown(result.final_output))


## Notes

- **Geocoding is automatic!** You can use location names like "Nairobi, Kenya" or "New York, USA"
- The tools will automatically convert location names to coordinates using Open-Meteo's free geocoding API
- You can still use coordinates directly if you prefer
- Make sure you have `STORMGLASS_API_KEY` set in your `.env` file
- Make sure you have `PUSHOVER_USER` and `PUSHOVER_TOKEN` set for push notifications
- Forecasts can be requested for up to 240 hours (10 days)
- Marine weather data is particularly useful for coastal locations
- Check your Pushover app for notifications about completed queries

## Check Traces

View detailed traces of the agent's weather queries:

https://platform.openai.com/traces
